In [1]:
import tensorflow as tf
import numpy as np 
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.cross_validation import train_test_split
from numpy.linalg import inv
housing = fetch_california_housing()

/home/ting/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def normal_eq(X, y):
    return inv(X.T.dot(X)).dot(X.T).dot(y)

In [3]:
df = pd.DataFrame(data = housing.data,columns=housing.feature_names)
dff = df.apply(lambda x: (x-x.mean())/x.std(), axis=0).fillna(0)
dff.insert(loc = 0 ,column='label',value=housing.target)

In [4]:
X , y =dff.iloc[:,1:].values,dff.iloc[:,0].values
X = np.column_stack((np.ones(len(X)), X))
X_train , X_test ,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state = 0)
y_train = y_train.reshape(y_train.shape[0],1)
y_test = y_test.reshape(y_test.shape[0],1)
n_dim = X_train.shape[1]
X_test.shape

(2064, 9)

In [5]:
X = tf.placeholder(tf.float32, [None,n_dim], name='x')
Y = tf.placeholder(tf.float32 , [None , 1],name = 'y_')
W = tf.matmul(tf.matrix_inverse(tf.matmul(tf.transpose(X),X)),tf.matmul(tf.transpose(X),Y))

In [6]:
init = tf.global_variables_initializer()

In [7]:
sess = tf.Session()
sess.run(init)

In [8]:
#print('Weights:',sess.run(W,feed_dict={X:X_train,Y:y_train}))
w = sess.run(W,feed_dict={X:X_train,Y:y_train})
print(w)

[[ 2.06835771]
 [ 0.82586974]
 [ 0.11848944]
 [-0.25575805]
 [ 0.29522195]
 [-0.006164  ]
 [-0.05498747]
 [-0.89618969]
 [-0.86659926]]


In [9]:
X_test = X_test.astype('float32')
pred_y = tf.matmul(X_test,w)
error_rate = tf.reduce_mean(abs((y_test - pred_y)) / y_test)
print(sess.run(error_rate))

0.320751
